# Analyse Cryptomonnaie avec Binance Vision

Ce notebook utilise Binance Vision pour télécharger de gros datasets historiques de cryptomonnaies.

In [30]:
# PARAMÈTRES DE CONFIGURATION

# Durée d'analyse (en jours) - Binance Vision est optimisé pour de grandes périodes
DUREE_JOURS = 1

# Intervalle des données
# Options: 1s, 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
INTERVAL = '1m'

# Symbole à analyser
SYMBOL = 'BTCUSDT'

print(f"Configuration:")
print(f"- Symbole: {SYMBOL}")
print(f"- Durée d'analyse: {DUREE_JOURS} jours")
print(f"- Intervalle: {INTERVAL}")
print(f"- Source: Binance Vision (données historiques)")
print()

Configuration:
- Symbole: BTCUSDT
- Durée d'analyse: 1 jours
- Intervalle: 1m
- Source: Binance Vision (données historiques)



In [31]:
# IMPORTS ET FONCTIONS

import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import os
import zipfile
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

def download_binance_vision_data(symbol=SYMBOL, interval=INTERVAL, days_back=DUREE_JOURS):
    """
    Télécharge les données historiques depuis Binance Vision pour de gros datasets
    """
    print(f"Utilisation de Binance Vision pour télécharger {symbol} - {interval}")
    print(f"Période: {days_back} jours")
    print()
    
    # Calculer les dates nécessaires
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    
    all_data = []
    current_date = start_date
    success_count = 0
    total_periods = 0
    
    while current_date <= end_date:
        # Format de date pour l'URL Binance Vision
        date_str = current_date.strftime('%Y-%m-%d')
        
        # URL pour les données journalières
        url = f"https://data.binance.vision/data/spot/daily/klines/{symbol}/{interval}/{symbol}-{interval}-{date_str}.zip"
        
        print(f"Téléchargement: {date_str}", end=" ")
        
        try:
            response = requests.get(url, timeout=30)
            
            if response.status_code == 200:
                # Extraire le fichier CSV du ZIP
                with zipfile.ZipFile(BytesIO(response.content)) as zip_file:
                    csv_filename = f"{symbol}-{interval}-{date_str}.csv"
                    if csv_filename in zip_file.namelist():
                        csv_content = zip_file.read(csv_filename)
                        
                        # Lire le CSV
                        df_day = pd.read_csv(BytesIO(csv_content), header=None, names=[
                            'timestamp', 'open', 'high', 'low', 'close', 'volume',
                            'close_time', 'quote_asset_volume', 'number_of_trades',
                            'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
                        ])
                        
                        all_data.append(df_day)
                        success_count += 1
                        total_periods += len(df_day)
                        print(f"✓ ({len(df_day)} périodes)")
                    else:
                        print(f"⚠ Fichier CSV non trouvé")
            else:
                print(f"⚠ Non disponible (HTTP {response.status_code})")
                
        except requests.exceptions.RequestException as e:
            print(f"✗ Erreur réseau")
        except zipfile.BadZipFile:
            print(f"✗ ZIP corrompu")
        except Exception as e:
            print(f"✗ Erreur: {str(e)[:50]}")
        
        current_date += timedelta(days=1)
        time.sleep(0.1)  # Pause pour éviter la surcharge
    
    print(f"\nRésumé du téléchargement:")
    print(f"- Jours avec succès: {success_count}/{(end_date - start_date).days + 1}")
    print(f"- Total des périodes: {total_periods}")
    
    if all_data:
        # Combiner toutes les données
        combined_df = pd.concat(all_data, ignore_index=True)
        return combined_df.to_dict('records')
    else:
        print("Aucune donnée récupérée via Binance Vision")
        return []

def create_dataframe(data, symbol_name):
    if not data:
        print(f"Aucune donnée disponible pour {symbol_name}")
        return pd.DataFrame()
        
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Conversion des types
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)
    
    # Gestion des timestamps - Binance Vision utilise différents formats
    print("Conversion des timestamps...")
    timestamp_converted = False
    
    # Vérifier le format des timestamps en examinant quelques valeurs
    sample_timestamps = df['timestamp'].head(3).astype(str)
    print(f"Échantillon de timestamps: {list(sample_timestamps)}")
    
    try:
        # Essai avec millisecondes (format standard)
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        timestamp_converted = True
        print("✓ Timestamps convertis depuis millisecondes")
    except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
        try:
            # Essai avec microsecondes (nouveau format depuis janvier 2025)
            print("⚠ Tentative conversion depuis microsecondes...")
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='us')
            timestamp_converted = True
            print("✓ Timestamps convertis depuis microsecondes")
        except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
            try:
                # Essai avec secondes
                print("⚠ Tentative conversion depuis secondes...")
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
                timestamp_converted = True
                print("✓ Timestamps convertis depuis secondes")
            except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                try:
                    # Si les valeurs sont trop grandes, diviser par 1000000 (microsecondes vers secondes)
                    print("⚠ Tentative correction des timestamps (division par 1M)...")
                    df['timestamp'] = pd.to_datetime(df['timestamp'] / 1000000, unit='s')
                    timestamp_converted = True
                    print("✓ Timestamps corrigés et convertis")
                except (ValueError, pd.errors.OutOfBoundsDatetime, OverflowError):
                    print("✗ Impossible de convertir les timestamps")
    
    if not timestamp_converted:
        print(f"✗ Erreur: Impossible de traiter les timestamps pour {symbol_name}")
        return pd.DataFrame()
    
    # Vérifier la plausibilité des dates
    min_date = df['timestamp'].min()
    max_date = df['timestamp'].max()
    
    if min_date.year < 2009 or max_date.year > 2030:
        print(f"⚠ Dates suspectes détectées: {min_date} à {max_date}")
    
    df.set_index('timestamp', inplace=True)
    
    # Supprimer les doublons et trier
    df = df[~df.index.duplicated(keep='first')].sort_index()
    
    # Affichage des informations adaptées à l'intervalle
    if len(df) > 0:
        print(f"\nDonnées {symbol_name} traitées:")
        print(f"Période: {df.index.min().strftime('%Y-%m-%d %H:%M')} à {df.index.max().strftime('%Y-%m-%d %H:%M')}")
        print(f"Nombre de périodes ({INTERVAL}): {len(df):,}")
        
        # Calculs selon l'intervalle
        if INTERVAL.endswith('s'):
            seconds_per_period = int(INTERVAL[:-1])
            minutes = len(df) * seconds_per_period / 60
            hours = minutes / 60
            days = hours / 24
            print(f"Équivalent: {minutes:,.0f} minutes, {hours:,.1f} heures, {days:.1f} jours")
        elif INTERVAL.endswith('m'):
            minutes_per_period = int(INTERVAL[:-1])
            hours = len(df) * minutes_per_period / 60
            days = hours / 24
            print(f"Équivalent: {hours:,.1f} heures, {days:.1f} jours")
        elif INTERVAL.endswith('h'):
            hours_per_period = int(INTERVAL[:-1])
            hours = len(df) * hours_per_period
            days = hours / 24
            print(f"Équivalent: {hours:,.1f} heures, {days:.1f} jours")
        elif INTERVAL == '1d':
            print(f"Équivalent: {len(df)} jours")
        
        print(f"Prix minimum: ${df['low'].min():.2f}")
        print(f"Prix maximum: ${df['high'].max():.2f}")
        print(f"Volume total: {df['volume'].sum():,.2f} {symbol_name.replace('USDT', '')}")
    
    return df

print("Fonctions chargées avec succès.")

Fonctions chargées avec succès.


In [43]:
# TÉLÉCHARGEMENT DES DONNÉES

print(f"=== Récupération {SYMBOL} via Binance Vision ===\n")

# Téléchargement des données
btc_data = download_binance_vision_data(SYMBOL, INTERVAL, DUREE_JOURS)

# Création du DataFrame
df_btc = create_dataframe(btc_data, SYMBOL)

if len(df_btc) > 0:
    print(f"\nPremières lignes {SYMBOL} (intervalle {INTERVAL}):")
    print(df_btc.head())
    
    print(f"\nDernières lignes:")
    print(df_btc.tail())
else:
    print(f"\n⚠ Aucune donnée disponible pour {SYMBOL}")
    print("Suggestions:")
    print("- Vérifiez que le symbole est correct (ex: BTCUSDT, ETHUSDT)")
    print("- Essayez avec une période plus récente")
    print("- Certains intervalles peuvent ne pas être disponibles pour toutes les dates")

=== Récupération BTCUSDT via Binance Vision ===

Utilisation de Binance Vision pour télécharger BTCUSDT - 1m
Période: 1 jours

Téléchargement: 2025-09-13 ✓ (1440 périodes)
Téléchargement: 2025-09-14 ✓ (1440 périodes)
Téléchargement: 2025-09-14 ⚠ Non disponible (HTTP 404)

Résumé du téléchargement:
- Jours avec succès: 1/2
- Total des périodes: 1440
Conversion des timestamps...
Échantillon de timestamps: ['1757721600000000', '1757721660000000', '1757721720000000']
⚠ Tentative conversion depuis microsecondes...
✓ Timestamps convertis depuis microsecondes

Données BTCUSDT traitées:
Période: 2025-09-13 00:00 à 2025-09-13 23:59
Nombre de périodes (1m): 1,440
Équivalent: 24.0 heures, 1.0 jours
Prix minimum: $115127.27
Prix maximum: $116298.78
Volume total: 8,269.40 BTC

Premières lignes BTCUSDT (intervalle 1m):
                          open       high        low      close    volume  \
timestamp                                                                   
2025-09-13 00:00:00  116029.4

In [50]:
# VISUALISATION INTERACTIVE

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Paramètre de moyenne mobile (modifiable)
MA_PERIOD = 10

if len(df_btc) > 0:
    # Calculer la moyenne mobile sur les MA_PERIOD dernières périodes
    df_btc[f'ma_{MA_PERIOD}'] = df_btc['close'].rolling(window=MA_PERIOD).mean()
    # Décalage vers la gauche pour aligner (la fin des données n'est pas importante)
    decalage = MA_PERIOD // 2
    df_btc[f'ma_{MA_PERIOD}_shifted'] = df_btc[f'ma_{MA_PERIOD}'].shift(-decalage)

    # Calculer la différence entre close et la MA décalée
    df_btc['diff_close_ma'] = df_btc['close'] - df_btc[f'ma_{MA_PERIOD}_shifted']

    # Création de sous-graphiques: 3 lignes (prix+MA, différence, volume)
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.06,
        row_heights=[0.5, 0.25, 0.25],
        specs=[[{"secondary_y": False}],
               [{"secondary_y": False}],
               [{"secondary_y": False}]]
    )

    # Row 1: Graphique en ligne (close)
    fig.add_trace(
        go.Scatter(
            x=df_btc.index,
            y=df_btc['close'],
            mode='lines',
            name=f"{SYMBOL} Close",
            line=dict(color='#00c851', width=1)
        ),
        row=1, col=1
    )

    # Row 1: Tracer la moyenne mobile (décalée)
    fig.add_trace(
        go.Scatter(
            x=df_btc.index,
            y=df_btc[f'ma_{MA_PERIOD}_shifted'],
            mode='lines',
            name=f"MA {MA_PERIOD} (shifted)",
            line=dict(color='blue', width=1)
        ),
        row=1, col=1
    )

    # Row 2: Tracer la différence (Close - MA) seule
    fig.add_trace(
        go.Scatter(
            x=df_btc.index,
            y=df_btc['diff_close_ma'],
            mode='lines',
            name='Close - MA (shifted)',
            line=dict(color='red', width=1)
        ),
        row=2, col=1
    )

    # Ajouter une ligne zéro pour référence dans le subplot 2
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.6, row=2, col=1)

    # Row 3: Graphique des volumes
    fig.add_trace(
        go.Bar(
            x=df_btc.index,
            y=df_btc['volume'],
            name="Volume",
            marker_color='rgba(0, 150, 255, 0.6)',
            marker_line_color='rgba(0, 150, 255, 0.8)',
            marker_line_width=0.5
        ), 
        row=3, col=1,
    )

    # Configuration du layout
    fig.update_layout(
        title={
            'text': f"{SYMBOL} - Données Binance Vision ({INTERVAL}, {DUREE_JOURS} jours)",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 16}
        },
        height=900,
        showlegend=True,
        hovermode='x unified',
        xaxis_rangeslider_visible=False,
        template="plotly_white",
        margin=dict(l=50, r=50, t=80, b=50),
    )

    # Configuration des axes
    fig.update_yaxes(title_text="Prix (USD)", row=1, col=1)
    fig.update_yaxes(title_text="Close - MA (USD)", row=2, col=1)
    fig.update_yaxes(title_text="Volume", row=3, col=1)

    # Affichage
    config = {
        'displayModeBar': True,
        'displaylogo': False,
        'modeBarButtonsToRemove': ['pan2d', 'lasso2d']
    }

    print(f"Affichage du graphique {SYMBOL} avec {len(df_btc):,} périodes...")

    # Essayer une liste de renderers connus et sûrs; si tous échouent, tenter le comportement par défaut
    preferred_renderers = ['vscode', 'notebook', 'notebook_connected', 'iframe', 'colab', 'jupyterlab', 'svg', 'png']
    shown = False
    for r in preferred_renderers:
        try:
            print(f"Tentative d'affichage avec le renderer: {r}")
            fig.show(config=config, renderer=r)
            shown = True
            print(f"Affiché avec succès en utilisant le renderer: {r}")
            break
        except Exception as e:
            print(f"Renderer {r} a échoué: {e}")

    if not shown:
        try:
            print("Aucun renderer préféré n'a fonctionné; tentative avec fig.show() par défaut...")
            fig.show(config=config)
        except Exception as e:
            msg = str(e)
            print(f"Erreur lors de fig.show(): {msg}")
            # Si l'erreur mentionne Kaleido, tenter explicitement 'notebook' comme dernier recours
            if 'Kaleido' in msg or 'kaleido' in msg:
                try:
                    print("Erreur Kaleido détectée; tentative d'un renderer 'notebook' en fallback...")
                    fig.show(config=config, renderer='notebook')
                    shown = True
                except Exception as e2:
                    print(f"Fallback 'notebook' a également échoué: {e2}")
            else:
                print("Aucun renderer de repli n'a réussi.")

    if not shown:
        print("Impossible d'afficher la figure avec les renderers testés.")
else:
    print("Impossible d'afficher le graphique: aucune donnée disponible.")

Affichage du graphique BTCUSDT avec 1,440 périodes...
Tentative d'affichage avec le renderer: vscode


Affiché avec succès en utilisant le renderer: vscode
